In [91]:
import pandas as pd
import datetime
import calendar
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Import closing days defined in the excel file
closing_days = pd.read_excel('Calendar.xlsx')

# /* ****************************** GLOBAL MACRO-VARIABLES ******************************* */
# Dates

# Date used to indicate an error occurred
errdate = datetime.datetime.strptime('22001231','%Y%m%d')
# Get current year, month, day and hour
cal_yr = datetime.datetime.now().year
cal_mth = datetime.datetime.now().month
cal_day = datetime.datetime.now().day
hour = datetime.datetime.now().hour
# Get the format to yyyymm -> 2016*100 + 6 = 201600 + 6 = 201606
cal_yrm = cal_yr * 100 + cal_mth
cal_ymd = cal_yrm * 100 + cal_day
monthend = 0 # Month end is defined so that some parts of code will be run on month end only and other parts of code will not

# /********************/
# /* Accounting month */
# /********************/
# /* This is to find the current and previous accounting month as well as to determine
#    if the current day should be considered as month end. For example:
#    	   If we have: Current month = 2016/08 and Closing day = 2016/08/25
#        	   1) Current day = 2016/08/04:
# 				Accounting month => 2016/08	
# 				Last Closing(LC) => 2016/07
# 				Month End 		 => 0 (NO)
#        	   2) Current day = 2016/08/26:

# 				Accounting month => 2016/08	(Before Noon) 
# 				Last Closing(LC) => 2016/07	(Before Noon)
# 				Month End 		 => 1 (YES)	(Before Noon) 

# 							 	 vs.

# 				Accounting month => 2016/09	(After Noon)
# 				Last Closing(LC) => 2016/08	(After Noon)
# 				Month End 		 => 0 (NO)	(After Noon)

#        	   3) Current day = 2016/08/29:
# 				Accounting month => 2016/09	
# 				Last Closing(LC) => 2016/08
# 				Month End 		 => 0 (NO)
# */

# 	/* Work with dataset "CLOSING_DAYS" imported from Excel. yrm and date_of_closing 
# 	   comes from CLOSING_DAYS in Excel, this only selects the row where it is the 
# 	   current accounting month 

# For current accounting period
incyr = 0
incrmth = 0
# For previous accounting period
incryr_lc = 0
incrmth_lc = 0

date_of_closing = closing_days.loc[closing_days['yrm'] == cal_yrm,'date_of_closing'].iloc[0]
if ((cal_day <= date_of_closing.day) | ((cal_day == date_of_closing.day) & (hour <= 23))):
    # Month end occurs the day after the closing date
    if (cal_day == date_of_closing.day + 1):
        monthend = 1
    else: montheend = 0
    if cal_mth == 1:
        incryr_lc = -1
        incrmth_lc = 11
    else: incrmth_lc = -1
else:
    if cal_mth == 12:
        incryr = 1
        incrmth = -11
    else: incrmth = 1

acc_yr = incryr + cal_yr
acc_mth = incrmth + cal_mth
acc_yrm = (incryr + cal_yr) * 100 + (incrmth + cal_mth)
lc_yr = incryr_lc + cal_yr
lc_mth = incrmth_lc + cal_mth
lc_yrm = (incryr_lc + cal_yr) * 100 + (incrmth_lc + cal_mth)

